In [1]:
# Imports

# Standard Python Data Science imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Pytorch imports
import torch
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchtext.data.utils import get_tokenizer

In [2]:
# Partly taken from https://github.com/prakashpandey9/Text-Classification-Pytorch
class SelfAttention(nn.Module):
    def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):
        super(SelfAttention, self).__init__()

        """
        Arguments
        ---------
        batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
        output_size : 2 = (pos, neg)
        hidden_sie : Size of the hidden_state of the LSTM
        vocab_size : Size of the vocabulary containing unique words
        embedding_length : Embeddding dimension of GloVe word embeddings
        weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table 
        
        --------
        
        """

        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embedding_length = embedding_length
        self.weights = weights

        self.word_embeddings = nn.Embedding(vocab_size, embedding_length)
        self.word_embeddings.weights = nn.Parameter(weights, requires_grad=False)
        self.dropout = 0.8
        self.bilstm = nn.LSTM(embedding_length, hidden_size, dropout=self.dropout, bidirectional=True)
        # We will use da = 350, r = 30 & penalization_coeff = 1 
        # as per given in the self-attention original ICLR paper
        self.W_s1 = nn.Linear(2*hidden_size, 350)
        self.W_s2 = nn.Linear(350, 30)
        self.fc_layer = nn.Linear(30*2*hidden_size, 2000)
        self.label = nn.Linear(2000, output_size)

    def attention_net(self, lstm_output):

        """
        Now we will use self attention mechanism to produce a matrix 
        embedding of the input sentence in which every row represents an
        encoding of the input sentence but giving an attention to a 
        specific part of the sentence. We will use 30 such embedding of 
        the input sentence and then finally we will concatenate all the 30 
        sentence embedding vectors and connect it to a fully connected layer 
        of size 2000 which will be connected to the output layer of size 2 
        returning logits for our two classes i.e., pos & neg.
        
        Arguments
        ---------
        lstm_output = A tensor containing hidden states corresponding to each time step of the LSTM network.
        ---------
        Returns : Final Attention weight matrix for all the 30 different sentence embedding in which each of 30 embeddings give
                  attention to different parts of the input sentence.
        Tensor size : lstm_output.size() = (batch_size, num_seq, 2*hidden_size)
                      attn_weight_matrix.size() = (batch_size, 30, num_seq)
        """
        attn_weight_matrix = self.W_s2(F.tanh(self.W_s1(lstm_output)))
        attn_weight_matrix = attn_weight_matrix.permute(0, 2, 1)
        attn_weight_matrix = F.softmax(attn_weight_matrix, dim=2)

        return attn_weight_matrix

    def forward(self, input_sentences, batch_size=None, return_attn=False):

        """ 
        Parameters
        ----------
        input_sentence: input_sentence of shape = (batch_size, num_sequences)
        batch_size : default = None. 
        Used only for prediction on a single sentence after training (batch_size = 1)
        return_attn : bool determining whether to return attention layer activation 
                      default = False
        
        Returns
        -------
        Output of the linear layer containing logits for pos & neg class.
        Attention layer if the return_attn is set
        
        """

        input = self.word_embeddings(input_sentences)
        input = input.permute(1, 0, 2)
        if batch_size is None:
            h_0 = Variable(torch.zeros(2, self.batch_size, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(2, self.batch_size, self.hidden_size).cuda())
        else:
            h_0 = Variable(torch.zeros(2, batch_size, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(2, batch_size, self.hidden_size).cuda())

        output, (h_n, c_n) = self.bilstm(input, (h_0, c_0))
        output = output.permute(1, 0, 2)
        # output.size() = (batch_size, num_seq, 2*hidden_size)
        # h_n.size() = (1, batch_size, hidden_size)
        # c_n.size() = (1, batch_size, hidden_size)
        attn_weight_matrix = self.attention_net(output)
        # attn_weight_matrix.size() = (batch_size, r, num_seq)
        # output.size() = (batch_size, num_seq, 2*hidden_size)
        hidden_matrix = torch.bmm(attn_weight_matrix, output)
        # hidden_matrix.size() = (batch_size, r, 2*hidden_size)
        # Let's now concatenate the hidden_matrix and connect it to the fully connected layer.
        fc_out = self.fc_layer(hidden_matrix.view(-1, hidden_matrix.size()[1]*hidden_matrix.size()[2]))
        logits = self.label(fc_out)
        # logits.size() = (batch_size, output_size)
        if not return_attn:
            return logits
        else:
            return logits, attn_weight_matrix

In [109]:
def read_embeddings(filename, vocab_size=10000, compute_avg=True):
    """
    Utility function, loads in the `vocab_size` most common embeddings from `filename`
  
    Arguments:
      - filename:     path to file
                      automatically infers correct embedding dimension from filename
      - vocab_size:   maximum number of embeddings to load
      - compute_avg:  bool to decide whether to comnpute the average word embedding,
                      which can be used as a <unk> embedding

      Returns 
      - embeddings:   torch.FloatTensor matrix of size (vocab_size x word_embedding_dim)
      - vocab:        dictionary mapping word (str) to index (int) in embedding matrix
    """

    # get the embedding size from the first embedding
    with open(filename, encoding="utf-8") as file:
        word_embedding_dim = len(file.readline().split(" ")) - 1

    vocab = {}
    inv_vocab = {}
    
    if compute_avg:
        # Add extra embedding for <unk> and <pad>
        # last index is <unk>, first index is <pad>
        embeddings = np.zeros((vocab_size + 2, word_embedding_dim))
    else:
         # Only add extra embedding for <pad>
         # first index is <pad>
        embeddings = np.zeros((vocab_size + 1, word_embedding_dim))
        

    with open(filename, encoding="utf-8") as file:
        for idx, line in enumerate(file):

            if idx >= vocab_size:
                break
            
            cols = line.rstrip().split(" ")
            val = np.array(cols[1:])
            word = cols[0]
            embeddings[idx + 1] = val
            vocab[word] = idx + 1
            inv_vocab[idx + 1] = word
    
    # Set <unk> embedding to the average of all other embedding vects in vocab
    if compute_avg:
        embeddings[-1] = embeddings[:-1].mean(axis=0)
    
    return torch.FloatTensor(embeddings), vocab, inv_vocab

In [124]:
def sent_embedd(sent, model, vocab_dict, tokenizer, max_length, unk_embedd=True):
    '''
    Helper function to extract the embedding for an input sentence.
    '''
    idxs = [0 for i in range(max_length)]
    i = 0
    for word in tokenizer(sent):
        if i < max_length:
            if word in vocab_dict:
                idxs[i] = vocab_dict[word]
            else:
                # If using <unk> embedding, append 
                # the final index where that embedding is stored
                if unk_embedd:
                    idxs[i] = len(vocab_dict)
            i += 1
    return torch.LongTensor([idxs]).to(device)
            
def sent_pred(sent, model, vocab_dict, max_length):
    '''
    Runs the model on an input sentence.
    
    Arguments: 
    
      sent : str. The input sentence.
      model : the pytorch model to be used.
      vocab_dict : dict. A dictionary with words as keys and their indices as values
     
    Returns:
      pred : np array. The prediction, wich is a normalized array with a value for \
             each class, representing the predicted probability for that class
      attns : the attention matrix
    '''
    input_tensor = sent_embedd(sent, model, vocab_dict, max_length)
    pred, attns = model(input_tensor, return_attn=True)
    return pred.detach().cpu().numpy(), attns

In [150]:
def train_single(input_tensor_batch, target_tensor_batch, model, 
          model_optimizer, criterion):
    '''
    A single forward and backward pass of the neural net on a single training batch.
    '''
    target_tensor = torch.stack(target_tensor_batch).reshape(len(input_tensor_batch))
    input_tensor = torch.stack(input_tensor_batch)
    input_tensor = input_tensor.reshape(len(input_tensor_batch), input_tensor.shape[2])
    output = model(input_tensor, return_attn=False)
    loss = criterion(output, target_tensor)
    loss.backward()
    model_optimizer.step()
    return loss.item()

def train(input_tensors, target_tensors, model, model_optimizer, criterion, n_epochs):
    '''
    Train the attention classfier for a given number of epochs on the whole training set.
    '''
    losses = []
    for epoch in range(n_epochs):
        loss = 0
        for i in range(len(input_tensors)):
            input_tensor = input_tensors[i]
            target_tensor = target_tensors[i]
            loss += train_single(input_tensor, target_tensor, model, 
                                 model_optimizer, criterion)
        print(f"Epoch {epoch} : Loss {loss/len(input_tensors):.4f}")
        losses.append(loss/len(input_tensors))
    return losses

In [126]:
unk_embedd = True # Bool for whether to create an embedding for the <unk> token
# Get initial word vector embedings from disk
glove_filename = './GloveEmbeddings/glove.6B.100d.txt' 
glove_embeddings, vocab_dict, inv_vocab_dict = read_embeddings(glove_filename, 
                                                               vocab_size=50000,
                                                               compute_avg=unk_embedd)
# Create tokenizer
tokenizer = get_tokenizer('basic_english')

In [169]:
# Set dimensions and hyperparameters
batch_size = 16
output_size = 2
hidden_size = 100
vocab_size = glove_embeddings.shape[0]
embedding_length = glove_embeddings.shape[1]
learning_rate = 10e-5
max_length = 40 # max sentence length (in tokens)
# Initialize embedding weights
weights = glove_embeddings
# Create model
model = SelfAttention(batch_size, output_size, hidden_size, vocab_size, 
                      embedding_length, weights)
#Define model optimizer
model_optimizer = optim.SGD(model.parameters(), lr=learning_rate)
# Use cross entropy loss
criterion = nn.CrossEntropyLoss()
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

SelfAttention(
  (word_embeddings): Embedding(50002, 100)
  (bilstm): LSTM(100, 100, dropout=0.8, bidirectional=True)
  (W_s1): Linear(in_features=200, out_features=350, bias=True)
  (W_s2): Linear(in_features=350, out_features=30, bias=True)
  (fc_layer): Linear(in_features=6000, out_features=2000, bias=True)
  (label): Linear(in_features=2000, out_features=2, bias=True)
)

In [8]:
# pred, attns = sent_pred('the world is red', model, vocab_dict)
# print(pred)
# print(attns.sum(axis=1).reshape(attns.shape[2]).detach().cpu().numpy())

In [129]:
train_file = "../cross_style_transfer_internal/data/xslue/StanfordPoliteness/train.tsv"
data = pd.read_csv(train_file, names=['domain', 'id', 'text', 'score'], sep='\t')

In [156]:
def parse_stanford_politeness(polite_df):
    '''
    Parse stanford politeness dataframe into the format we need for classification.
    '''
    input_df = pd.DataFrame()
    input_df['text'] = polite_df['text']
    # Map scores >= 0 (polite) to label 1 and scores < 0 (impolite) to label 0.
    input_df['label'] = polite_df['score'].apply(lambda x : int(x >= 0))
    return input_df

def df_to_training_pairs(df, batch_size, max_length):
    input_tensors = df['text'].apply(lambda x : sent_embedd(x, model, vocab_dict, tokenizer, max_length))
    target_tensors = df['label'].apply(lambda x : torch.LongTensor([x]).to(device))
    return input_tensors.values.reshape(-1, batch_size).tolist(), target_tensors.values.reshape(-1, batch_size).tolist()

In [157]:
input_df = parse_stanford_politeness(data)

In [158]:
input_df.head()

,text,label
0,Where did you learn English? How come you're t...,0
1,Thanks very much for your edit to the <url> ar...,1
2,Sir i think u hav many friends on wiki who can...,0
3,I can't find it. Maybe I didn't manage to gue...,1
4,"I can't spend too much time, and I'm no specia...",1


In [31]:
# Testing a single pass on a single input

# sent = input_df['text'].iloc[0]
# target_tensor = torch.LongTensor([input_df['label'].iloc[0]]).to(device)
# input_tensor = sent_embedd(sent, model, vocab_dict)
# train_single(input_tensor, target_tensor, model, 
#           model_optimizer, criterion)

In [170]:
input_tensors, target_tensors = df_to_training_pairs(input_df.head((len(input_df)//batch_size)*batch_size), 
                                                                   batch_size, max_length)

In [172]:
train(input_tensors, target_tensors, model, model_optimizer, criterion, 20)

Epoch 0 : Loss 0.7273
Epoch 1 : Loss 0.7626
Epoch 2 : Loss 0.7861
Epoch 3 : Loss 0.7671
Epoch 4 : Loss 0.9012
Epoch 5 : Loss 0.7876
Epoch 6 : Loss 0.7040
Epoch 7 : Loss 0.7569
Epoch 8 : Loss 0.7684
Epoch 9 : Loss 0.8150
Epoch 10 : Loss 0.8577
Epoch 11 : Loss 0.9584
Epoch 12 : Loss 0.9595
Epoch 13 : Loss 0.9125
Epoch 14 : Loss 0.8147
Epoch 15 : Loss 0.7696
Epoch 16 : Loss 0.7491
Epoch 17 : Loss 0.6014
Epoch 18 : Loss 0.5949
Epoch 19 : Loss 0.5755


[0.7272814550957123,
 0.7625551431306771,
 0.7861400538353951,
 0.7671344040759972,
 0.9012486181379139,
 0.7876155763477474,
 0.7039959943526751,
 0.7568828998157731,
 0.7684298430557375,
 0.8150115689480459,
 0.8576596030941257,
 0.9583921643917437,
 0.9594507070814634,
 0.912521401589567,
 0.8147297311912883,
 0.7696443870905545,
 0.7490790940385748,
 0.6014387190535471,
 0.5948914579031143,
 0.5755344770074665]

To do:
- [X] handle unk words better
    - currently initalizing to the average of all word embeddings like suggested [here](https://stackoverflow.com/questions/49239941/what-is-unk-in-the-pretrained-glove-vector-files-e-g-glove-6b-50d-txt)
- [X] make work with batches
- [ ] use different, better embeddings
- [X] use better tokenizer, like spacy or some huggingface transformer model
- [ ] train and save a good model
- [ ] visualize attentions
- [ ] make work with other datasets
- [ ] convert to .py script that runs with input file that determines which data and parameters to use